In [ ]:
from tqdm import notebook
!pip install selenium
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# 엑셀 파일 리뷰 부분 전처리하는 함수(.xlsx 제외하고 입력)
def do_preprocess(file_name):
    df = pd.read_excel(file_name + ".xlsx")
    not_using_text_list = ["한줄평", "병원리뷰", r"[0-9]."]
    
    for i in range(len(df)):
        temp = df.loc[i, '리뷰']
        temp = preprocess(temp)
        
        for not_using_text in not_using_text_list:
            temp = remove_text(temp, not_using_text)
            
        df.loc[i, '리뷰'] = temp
        
    df.to_excel(file_name + "전처리" + ".xlsx")

    
# 텍스트 전처리하는 함수
def preprocess(text):
    text= re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", ".", text)
    text = re.sub(r"[\`\~\@\#\$\%\^\&\*\(\)\-\_\=\+\<\>\;\:\'\"\[\]\{\}\\\/]", " ", text)
    text = re.sub(r"[^가-힣A-Za-z0-9\.\?\!]", " ", text)
    text = re.sub(r"\s*[\.]+", ".", text)
    text = re.sub(r"\s*[\?]+", "?", text)
    text = re.sub(r"\s*[\!]+", "!", text)
    text = re.sub(r"\s+", " ", text)
    text.strip()
    
    return text


# 불용어 제거하는 함수
def remove_text(original_text, not_using_text):
    result = re.sub(not_using_text, "", original_text)
    
    return result

In [ ]:
def naver_place_review(driver, url, hospital, point):
    try:
        driver.get(url)
        # print('연결성공')
        result = []
        try:
            try:
                hospitalUrl = driver.find_elements(By.XPATH,
                                                   '//*[@id="loc-main-section-root"]/section/div/ul/li[contains(.,' + hospital + ')]')
            except:
                hospitalUrl = driver.find_elements(By.XPATH,
                                                   '//*[@id="loc-main-section-root"]/section/div/ul/li')
            if len(hospitalUrl) == 0:
                url = 'https://m.place.naver.com/place/list?level=top&entry=pll&x=null&y=null&query=' + hospital
                driver.get(url)
                try:
                    hospitalUrl = driver.find_element(By.XPATH,
                                                      '//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li/div[2]/a[1][contains(.,' + hospital + ')]')
                except:
                    hospitalUrl = driver.find_element(By.XPATH,
                                                      '//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li/div[2]/a[1]')
                reviewDtail = hospitalUrl.get_attribute('href').split('/')[-1].split('?')[0]
                reviewUrl = 'https://pcmap.place.naver.com/hospital/' + reviewDtail + '/review/visitor'
                https://pcmap.place.naver.com/hospital/12975400/review/visitor
                script = f"window.open('{reviewUrl}');"
                driver.execute_script(script)
                driver.switch_to.window(driver.window_handles[-1])
                # time.sleep(3)
                moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                while len(moreButton) > 0:
                    moreButton[0].click()
                    time.sleep(1)
                    moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                try:
                    comentMoreBtn = driver.find_elements(By.CSS_SELECTOR, 'span._3_09q')
                    print(len(comentMoreBtn))
                    if len(comentMoreBtn) > 0:
                        for btn in comentMoreBtn:
                            driver.execute_script('arguments[0].click();', btn)
                            # btn.click()
                except:
                    print('없음')
                try:
                    contents = driver.find_elements(By.CSS_SELECTOR, 'li._3l2Wz')
                    for review in contents:
                        content = {
                            'review': review.find_element(By.CSS_SELECTOR, 'div.faZHB').text,
                            'date': review.find_element(By.XPATH, '//div[3]/span[1]/span[2]').text
                        }
                        result.append(content)
                except:
                    print('없음')
                # print(result)
            else:
                for i in hospitalUrl:
                    if i.text.find(point) != -1:
                        reviewDtail = i.get_attribute('data-loc_plc-doc-id')
                        reviewUrl = 'https://pcmap.place.naver.com/hospital/' + reviewDtail + '/review/visitor'
                        script = f"window.open('{reviewUrl}');"
                        driver.execute_script(script)
                        driver.switch_to.window(driver.window_handles[1])
                        # time.sleep(3)
                        moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                        while len(moreButton) > 0:
                            moreButton[0].click()
                            time.sleep(1)
                            moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                        try:
                            comentMoreBtn = driver.find_elements(By.CSS_SELECTOR, 'span._3_09q')
                            print(len(comentMoreBtn))
                            if len(comentMoreBtn) > 0:
                                for btn in comentMoreBtn:
                                    driver.execute_script('arguments[0].click();', btn)
                                    # btn.click()
                        except:
                            print('없음')
                        try:
                            contents = driver.find_elements(By.CSS_SELECTOR, 'li._3l2Wz')
                            for review in contents:
                                content = {
                                    'review': review.find_element(By.CSS_SELECTOR, 'div.faZHB').text,
                                    'date': review.find_element(By.XPATH, '//div[3]/span[1]/span[2]').text
                                }
                                result.append(content)
                        except:
                            print('없음')
                        print(result)
                        break
            return result
        except Exception as e:
            print('tistory Error:', e)
            result = 0
            return result
    except Exception as e:
        print('사이트 오류')
        result = 0
    return result
